<a href="https://colab.research.google.com/github/bird0401/Instance_level_recognition/blob/main/01_data_download.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
import pickle
from tqdm import tqdm
import requests
import re
from collections import defaultdict
import shutil
import random

In [9]:
# !cp /content/drive/MyDrive/instance_level_recognition/fileurl_dog.pickle .
# !cp /content/drive/MyDrive/instance_level_recognition/fileurl_dog_02.pickle .
!cp /content/drive/MyDrive/instance_level_recognition/fileurl_car_02.pickle .
!cp /content/drive/MyDrive/instance_level_recognition/fileurl_uspresident.pickle .
!cp /content/drive/MyDrive/instance_level_recognition/fileurl_uspresident_02.pickle .
!cp /content/drive/MyDrive/instance_level_recognition/fileurl_tower.pickle .
!cp /content/drive/MyDrive/instance_level_recognition/fileurl_tower_02.pickle .

In [10]:
# urlを保存しているpickleファイルをロード
categories=["car2","uspresident","uspresident_02","tower","tower_02"]
category=categories[1]
# with open('/content/fileurl_car_02.pickle', 'rb') as f:
filename="/content/fileurl_"+category+".pickle"
with open(filename, 'rb') as f:
  fileurls = pickle.load(f)

In [11]:
# 画像の保存先ディレクトリ
save_image_dir = "imgs_"+category

In [12]:
!mkdir {save_image_dir}

mkdir: cannot create directory ‘imgs_uspresident’: File exists


In [15]:
# wikiにアクセスする際にユーザエージェントが必要
ua = "VisualEntityLinkingBot/1.0 (matsushima-hiromu427@g.ecc.u-tokyo.ac.jp) download-image/1.0"
headers = {"User-Agent": ua}

# ダウンロードに時間がかかるので分割
ids=list(fileurls.keys())
random.shuffle(ids)
start,end=0,100

In [16]:
# urlから画像をダウンロード

for id in tqdm(ids[start:end]):
    c = 0
    urls = fileurls[id]
    fail_cnt = 0

    for url in urls:
        c += 1
        # print("download: ", url)
        # ダウンロードしたファイルの命名は「entity+ファイル番号」
        image_name = "Q" + str(id) + "_" + str(c).zfill(3)
        extension = url.split(".")[-1]
        image_name += "." + extension

        # try download for 5 times
        flg = False
        for _ in range(5):
            res = requests.get(url, stream=True, headers=headers)
            if res.status_code == 200:
                flg = True
                with open(save_image_dir +"/"+ image_name, "wb") as file:
                    res.raw.decode_content = True
                    shutil.copyfileobj(res.raw, file)
                break

100%|██████████| 100/100 [12:15<00:00,  7.36s/it]


In [17]:
!ls imgs_car2|wc -l

ls: cannot access 'imgs_car2': No such file or directory
0


In [18]:
!tar -zcvf {save_image_dir}.tar.gz {save_image_dir}

imgs_uspresident/
imgs_uspresident/Q3536054_104.jpg
imgs_uspresident/Q1739972_069.JPG
imgs_uspresident/Q12046742_003.gif
imgs_uspresident/Q3502403_058.jpg
imgs_uspresident/Q1626684_052.JPG
imgs_uspresident/Q960745_011.jpg
imgs_uspresident/Q9163497_016.jpg
imgs_uspresident/Q737292_006.jpg
imgs_uspresident/Q91029115_002.jpg
imgs_uspresident/Q1140349_024.jpg
imgs_uspresident/Q7290047_017.JPG
imgs_uspresident/Q1858576_004.jpg
imgs_uspresident/Q3536054_011.jpg
imgs_uspresident/Q3502403_095.jpg
imgs_uspresident/Q7290047_006.jpg
imgs_uspresident/Q39632_020.jpg
imgs_uspresident/Q1921537_011.jpg
imgs_uspresident/Q90701201_009.jpg
imgs_uspresident/Q19481252_064.jpg
imgs_uspresident/Q91450545_004.jpg
imgs_uspresident/Q63636478_048.jpg
imgs_uspresident/Q63636478_055.jpg
imgs_uspresident/Q17064519_015.jpg
imgs_uspresident/Q605272_048.jpg
imgs_uspresident/Q1407097_008.jpg
imgs_uspresident/Q737292_043.jpg
imgs_uspresident/Q19481252_106.jpg
imgs_uspresident/Q605272_058.jpg
imgs_uspresident/Q55605945_0

In [19]:
!cp /content/{save_image_dir}.tar.gz /content/drive/MyDrive/instance_level_recognition

In [20]:
!rm -r {save_image_dir}
!rm {save_image_dir}.tar.gz
# !rm fileurl_dog.pickle